In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from config import api_key
from census import Census
import gmaps

c = Census(api_key, year = 2013)

# Output File (CSV)
output_data_file = "output_data/census.csv"

In [4]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", 
                          "B01003_001E", 
                          "B02001_001E", 
                          "B02008_001E", 
                          "B02009_001E", 
                          "B02011_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {"for": "zip code tabulation area: 63136, 63147, 63115, 63120," +\
                                                                             "63107, 63113, 63106, 63137,"+\
                                                                             "63125, 63111, 63123, 63118,"+\
                                                                             "63112, 63119, 63116, 63143,"+\
                                                                             "63104, 63109, 63139, 63103,"+\
                                                                             "63101, 63110, 63102, 63117,"+\
                                                                             "63133, 63108, 63130, 63105"})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B02001_001E": "Race",
                                      "B02008_001E": "White Identified", 
                                      "B02009_001E": "Black Identified",
                                      "B02011_001E": "Asian Identified",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zip Code"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zip Code", "Name", "Population", "Race", "White Identified", "Black Identified", "Asian Identified", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]
census_pd

,Zip Code,Name,Population,Race,White Identified,Black Identified,Asian Identified,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,63130,ZCTA5 63130,30810.0,30810.0,17389.0,12585.0,1560.0,55976.0,36054.0,4969.0,16.127881,5.066537
1,63133,ZCTA5 63133,8210.0,8210.0,558.0,7729.0,12.0,25239.0,13025.0,2815.0,34.287454,10.913520
2,63136,ZCTA5 63136,47220.0,47220.0,4834.0,42677.0,170.0,31051.0,16086.0,13019.0,27.570945,10.887336
3,63137,ZCTA5 63137,20546.0,20546.0,4458.0,16074.0,180.0,34019.0,16315.0,5275.0,25.674097,9.383822
4,63139,ZCTA5 63139,22212.0,22212.0,19132.0,2474.0,721.0,47280.0,30918.0,2667.0,12.007023,3.439582
5,63143,ZCTA5 63143,9996.0,9996.0,8165.0,1653.0,293.0,36210.0,25539.0,1905.0,19.057623,6.312525
6,63147,ZCTA5 63147,10534.0,10534.0,692.0,9800.0,20.0,29816.0,16969.0,2588.0,24.568065,10.375926
7,63101,ZCTA5 63101,2613.0,2613.0,1277.0,1284.0,62.0,54417.0,42701.0,610.0,23.344814,6.926904
8,63102,ZCTA5 63102,2216.0,2216.0,1012.0,1103.0,78.0,54018.0,32318.0,142.0,6.407942,2.572202
9,63103,ZCTA5 63103,6824.0,6824.0,3417.0,2961.0,402.0,34719.0,30274.0,1340.0,19.636577,5.216882


In [5]:
store_count = pd.read_csv(
    "zip_group.csv", dtype="object", encoding="utf-8")
store_count.head()

,Zip Code,Grocery Store Count
0,63141,5
1,63108,4
2,63118,4
3,63017,4
4,63116,3


In [6]:
df_merge = pd.merge(
    store_count, census_pd, how="right", on=["Zip Code", "Zip Code"])
df_merge.fillna(0)

,Zip Code,Grocery Store Count,Name,Population,Race,White Identified,Black Identified,Asian Identified,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,63108,4,ZCTA5 63108,21277.0,21277.0,11265.0,8178.0,1999.0,30157.0,31477.0,6409.0,30.121728,6.227382
1,63118,4,ZCTA5 63118,27350.0,27350.0,11041.0,15090.0,1153.0,28630.0,18570.0,8977.0,32.822669,8.405850
2,63116,3,ZCTA5 63116,44010.0,44010.0,30892.0,9897.0,3636.0,40251.0,23436.0,9080.0,20.631675,7.062031
3,63104,2,ZCTA5 63104,19520.0,19520.0,9122.0,10028.0,394.0,45498.0,29780.0,5417.0,27.751025,5.747951
4,63139,2,ZCTA5 63139,22212.0,22212.0,19132.0,2474.0,721.0,47280.0,30918.0,2667.0,12.007023,3.439582
5,63130,1,ZCTA5 63130,30810.0,30810.0,17389.0,12585.0,1560.0,55976.0,36054.0,4969.0,16.127881,5.066537
6,63119,1,ZCTA5 63119,33929.0,33929.0,29913.0,3261.0,1020.0,68348.0,40788.0,2518.0,7.421380,3.516166
7,63115,1,ZCTA5 63115,21059.0,21059.0,187.0,20918.0,30.0,26183.0,14527.0,6870.0,32.622632,11.111639
8,63117,1,ZCTA5 63117,9221.0,9221.0,7838.0,994.0,442.0,62593.0,48558.0,723.0,7.840798,2.624444
9,63113,1,ZCTA5 63113,12239.0,12239.0,395.0,11806.0,28.0,24561.0,14068.0,4533.0,37.037340,12.648092


In [7]:
overweight_df = pd.read_csv(
    "Rates of Overweight and Obesity by ZIP Code, City of St. Louis.csv", dtype="object", encoding="utf-8")
overweight_df.head()

,ZIP Code,Obese %,Overweight %,Overweight or Obese %
0,63136,40.50%,35.40%,76.00%
1,63147,39.60%,35.70%,75.30%
2,63115,38.90%,35.10%,74.00%
3,63120,39.50%,34.20%,73.60%
4,63107,38.30%,34.40%,72.70%


In [8]:
complete_df = pd.merge(
    df_merge, overweight_df, how="left", left_on = 'Zip Code', right_on = 'ZIP Code')
del complete_df['ZIP Code']
del complete_df['Name']
complete_df.fillna(0)

,Zip Code,Grocery Store Count,Population,Race,White Identified,Black Identified,Asian Identified,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Obese %,Overweight %,Overweight or Obese %
0,63108,4,21277.0,21277.0,11265.0,8178.0,1999.0,30157.0,31477.0,6409.0,30.121728,6.227382,16.90%,31.20%,48.10%
1,63118,4,27350.0,27350.0,11041.0,15090.0,1153.0,28630.0,18570.0,8977.0,32.822669,8.405850,27.80%,33.90%,61.70%
2,63116,3,44010.0,44010.0,30892.0,9897.0,3636.0,40251.0,23436.0,9080.0,20.631675,7.062031,25.20%,33.90%,59.00%
3,63104,2,19520.0,19520.0,9122.0,10028.0,394.0,45498.0,29780.0,5417.0,27.751025,5.747951,22.20%,35.20%,57.40%
4,63139,2,22212.0,22212.0,19132.0,2474.0,721.0,47280.0,30918.0,2667.0,12.007023,3.439582,22.40%,33.60%,56.00%
5,63130,1,30810.0,30810.0,17389.0,12585.0,1560.0,55976.0,36054.0,4969.0,16.127881,5.066537,12.30%,29.10%,41.40%
6,63119,1,33929.0,33929.0,29913.0,3261.0,1020.0,68348.0,40788.0,2518.0,7.421380,3.516166,31.30%,29.20%,60.40%
7,63115,1,21059.0,21059.0,187.0,20918.0,30.0,26183.0,14527.0,6870.0,32.622632,11.111639,38.90%,35.10%,74.00%
8,63117,1,9221.0,9221.0,7838.0,994.0,442.0,62593.0,48558.0,723.0,7.840798,2.624444,17.70%,33.70%,51.30%
9,63113,1,12239.0,12239.0,395.0,11806.0,28.0,24561.0,14068.0,4533.0,37.037340,12.648092,38.50%,34.10%,72.60%
